# Analyse slip velocity particle
In this notebook I analyse the slip velocity (difference fluid velocity and particle velocity)   
to make an estimation for the Reynolds partilcle number ($Re_p = U_{slip} d / \nu $). As  
pointed out by Irina, $Re_p$ might not be small and thus additional corrections to the Stokes  
drag force in the form of $\nu\rightarrow\nu_{eff}=\nu c(Re_p) = \nu(1+f(RE_p))$ should be added.       
To investigate this we measure the slip velocity in simulations with MR advection using:  
1. stokes drag
2. drag with  $c(Re_p)$ dependend drag   
And we do this for the slow manifold and full MR equation. We also investigate:   
1. variability $Re_p$
2. spin up time simulation (how mcuh time is needed before  slip velocity / $Re_p$ stabilizes)
32. effect of different timesteps


In [ ]:
# import needed packages
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from decimal import Decimal
from scipy.optimize import fsolve

from datetime import datetime, timedelta
from analysis_functions import trajectory_length,  make_PDF, Haversine, running_mean
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/simulations")
from helper import create_filelist

plt.style.use('../python_style_Meike.mplstyle')
# plotstyle:
color_array = np.array(['c','orange','purple','black'])
marker_array = np.array(['s','o','>','p'])
line_array = np.array(['-','--','-.',':'])
markerline_array = np.array(['-s','--o','-.>',':p'])



In [ ]:
# set needed constants
Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water

In [ ]:
# define functions
def Re_slip(Uslip, d, nu):
    """
    Calculates Reynolds number based on slip velocity particle. Has following dependencies:
    - d:  diameter particle particle (unit m)
    - Uslip: magnitude slip velocity (difference particle and fluid velocity) (unit m/s)
    - nu: kinematic velocity water (unit m^2/s)
    """
    return Uslip*d / nu


def calc_tidal_av(coordinates, window):
    """
    calculate (tidal) average of array (given by coordinates) over number of 
    array entries (for tides timesteps) given by window
    """
    cs = coordinates.cumsum(dim='obs',skipna=False)
    mean = (cs-cs.roll(obs=window))/float(window)#(cs[:,window:]-cs[:,:-window])#/float(window)
    return mean

def factor_drag_morrison2013(Rep):
    """
    Emperical stokes drag correction factor valid up to Rep < 10^6 from
    [1] F. A. Morrison, An introduction to fluid mechanics (2013)
    """
    c_REp = (1 + 
             Rep/24 * 2.6 * (Rep / 5.) / (1 + (Rep/5.)**(1.52))  +
             Rep/24 * 0.411 * (Rep / (2.63 * 10**5))**(-7.94) / (1 + (Rep / (2.63 * 10**5))**(-8)) +
             Rep/24 * 0.25 * (Rep) / ((10**6) + Rep  )
            )
    #tau_inv = 36 * nu * f_REp /( (1. + 2. * B) * L**2)
    return c_REp#tau_inv

def factor_drag_white1991(Rep):
    """
    stokes drag corection factor valid up to Rep < 10^5 from
    [1] F. M. White, Viscous fluid flow 2nd edition (1991)(p. 182)
    """
    c_REp = 1 + Rep / (4. * (1 +  np.sqrt(Rep))) + Rep / 60.
    return c_REp




In [ ]:
#compare drag factors

fig,ax = plt.subplots()
rep = np.logspace(-1,3,1000)
ax.plot(rep,factor_drag_morrison2013(rep),'-',color='k')
ax.plot(rep,factor_drag_white1991(rep),'--',color='grey')
ax.plot(rep,1+0.095*rep**0.8,'--',color='red')
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_xlabel('Re$_p$')
ax.set_ylabel('c(Re$_p$)')
ax.legend(['Morrison 2013','White 1991'])

print(factor_drag_white1991(457))
# we use white 1991 throughout as it is a way simpler expression and still does a very good job up to 10^5

# constant reynolds number
We perform simulation using drag force with $c(\mathrm{Re}_p)$ and measure the slip velocity  
we use this slip veloctity to calculated the measured $\mathrm{Re}_p$  
we compare the input $\mathrm{Re}_p$ with the measured $\mathrm{Re}_p$. The goal is to find  
the Reynolds number for which the input and measured $\mathrm{Re}_p$ are the same.  
We first use a short simulation of 24 hours where we save the slip velocity every 5 min and  
advect the particles using the slow manifold MR equation (more stable for high Reynolds numbers)  


## 24 hours sim with slow manifold

In [ ]:
# read in data
Replist_SM = np.array([1,10,50,300,400,450,500,1000,5000])
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_Rep_{Rep:04d}_slip_velocity.zarr')

#settings
runtime =  timedelta(days=1)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 2, 0, 0, 0, 0)
endtime = starttime + runtime
particle_type = 'inertial_SM_Rep_constant'

chunck_time = 100
nparticles = 88347
data_SM_Rep_constant_24h = {}
for Rep in Replist_SM: 
    file = file_base.format(loc=loc,
                           particle_type = particle_type,
                                                            y_s=starttime.year,
                                                            m_s=starttime.month,
                                                            d_s=starttime.day,
                                                            y_e=endtime.year,
                                                            m_e=endtime.month,
                                                            d_e=endtime.day,
                                                            B = int(B * 1000), 
                                                            tau = int(tau ),
                                                            land_handling = land_handling, 
                                                            cor_on = coriolis,
                                                            Rep = Rep )
    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)
    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_SM_Rep_constant_24h[Rep]=ds

In [ ]:
# plot selection of slip velocities
fig, ax = plt.subplots()
Replist_selection=np.array([300,500,1000,5000])
i=0
id=0
legend = []
for Rep in Replist_selection:
    ax.plot((data_SM_Rep_constant_24h[Rep].time[id,1:]-data_SM_Rep_constant_24h[Rep].time[id,0])/sec_in_hours,data_SM_Rep_constant_24h[Rep].uf[id,1:],marker_array[i],color=color_array[i])   
    legend.append('$\\mathrm{Re_p} =$' +  f'{Rep}')
    i+=1

ax.legend(legend)
ax.set_xlabel('time [hours]')
ax.set_ylabel('zonal slip velocity [m/s]')


In [ ]:
# make PDF and CDF for every particle Reynolds number
mean_SM_Rep_c_day={}
bins_SM_Rep_c_day={}
pdfs_SM_Rep_c_day={}
cdfs_SM_Rep_c_day={}

for Rep in Replist_SM:
    Re_particle =Re_slip(data_SM_Rep_constant_24h[Rep].Uslip,diameter, kinematic_viscosity_water)
    # fig, ax =plt.subplots()
    # ax.plot(Re_particle)
    # mean_Rep[particle_type] = Re_particle.mean(dim=None, skipna=True).values
    array = Re_particle.values.flatten()
    array=array[~np.isnan(array)]
    
    #somehow it does not understand that some of the values are nans?
    # test = data_SM_Rep_constant_24h[Rep].Uslip[:,2:].values.flatten()[~np.isnan(Rep)]
    
    # print(test[array>100000])#[array>100000])

    # array = array[array<100000]

    
    # array = array[array<100000]
    
    mean_SM_Rep_c_day[Rep]=np.mean(array)
    
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_SM_Rep_c_day[Rep]=bins
    pdfs_SM_Rep_c_day[Rep]=pdf
    cdfs_SM_Rep_c_day[Rep]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()

for Rep, color, marker in zip(Replist_selection, color_array, marker_array):
   ax.plot(bins_SM_Rep_c_day[Rep],pdfs_SM_Rep_c_day[Rep],marker,color=color)
ax.legend(np.char.add('Re$_p$ = ',  Replist_selection.astype(str)))
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
for Rep, color, marker in zip(Replist_selection, color_array, line_array):
   
   ax.plot(bins_SM_Rep_c_day[Rep],cdfs_SM_Rep_c_day[Rep],marker,color=color)
   i+=1 

ax.legend(np.char.add('Re$_p$ = ', Replist_selection.astype(str)))
ax.set_xscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot selection of trajectories for which slip velocities is bigger than threshold
Rep = 450
threshold = 0.1 # 0.1 # treshhold for slip velocity
above_threshold = data_SM_Rep_constant_24h[Rep].lon.where(data_SM_Rep_constant_24h[Rep].Uslip > threshold).notnull().any(dim='obs')

# Select trajectories that meet the condition
filtered_trajectories = data_SM_Rep_constant_24h[Rep].sel(trajectory=above_threshold)
# find index of maximum slip velocity for the selected trajectories
idmax= filtered_trajectories.Uslip.idxmax(dim='obs', skipna=True)

# plot selected trajectories and location of the maximum
fig,ax=plt.subplots(figsize=(30,8),
                     subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')

#plot selection trajectories
ax.plot(filtered_trajectories.lon.T,filtered_trajectories.lat.T,color='grey',linewidth=1);
#plot location maximum (not perse on land)
ax.plot(filtered_trajectories.lon.sel(obs =idmax).T,filtered_trajectories.lat.sel(obs =idmax).T,'o',color='red',markersize=5,markeredgewidth = 1, markeredgecolor = 'black');

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
# ax.set_xlim(-3.7,-2.5)
# ax.set_ylim(58.3,59.5)


### Estimate when measured and input particle reynolds number are the same
To estimate when the measured and input reynolds number are the same we use the  
fact that the slip velocity scales with $C(\mathrm{Re}_p^{\mathrm{in}})$, Thus we try   
to predict $\mathrm{Re}_p^{\mathrm{m}}$ using a single reference measurement with   
$\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{in}}$ resulting in $\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{m}}$  
Then we calculate the  $\mathrm{Re}_p^{\mathrm{measured}}$ using:   
$$  \mathrm{Re}_p^{\mathrm{m}}=\frac{\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{m}}  C(\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{in}})}{C(\mathrm{Re}_{p}^{\mathrm{in}}) }  $$

In [ ]:
# define functions needed to estimate the measured Rep based on a single reference measured
def Rep_measured(Rep_in, Rep_ref_in, Rep_ref_measured):
    return Rep_ref_measured * factor_drag_white1991(Rep_ref_in)/factor_drag_white1991(Rep_in)

def Rep_in_measured_root(Rep_in, Rep_ref_in, Rep_ref_measured):
   return Rep_measured(Rep_in, Rep_ref_in, Rep_ref_measured)-Rep_in

In [ ]:
# 

Repmeasured_SM = [mean_SM_Rep_c_day[Rep] for Rep in Replist_SM]
# find solution for when measured Rep is equal to input Rep
solution = [fsolve(Rep_in_measured_root, 450, args=(Repi, Repm))[0] for Repi,Repm in zip(Replist_SM,Repmeasured_SM)]
Rep_fit = np.array(solution).mean()

#plotting
fig, ax = plt.subplots()
# plot measured points
ax.plot(Replist_SM,Repmeasured_SM,'o',color='navy')
# plot predicted solution based on single value 
Reprange = np.logspace(-1,4,1000)
ax.plot(Reprange,Repmeasured_SM[5] * factor_drag_white1991(Replist_SM[5])/factor_drag_white1991(Reprange),'--',zorder=-5,color='navy')
#legend
ax.legend(['simulated Re$_p^m$','predicted Re$_p^m$'])
# plot fitted solution
ax.plot(Reprange,Reprange,color='grey',zorder=-10)
ax.text(85,60,'Re$_p^{in}$ = Re$_p^m$ = '+f'{Rep_fit:.2f}',color='grey',fontsize=15)
# label layout
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(0.1,10**4)
ax.set_ylim(0.1,10**4)
ax.set_xlabel('Re$_p^{in}$')
ax.set_ylabel('Re$_p^{m}$')

# #print solution
# print(f'Rep fit = {Rep_fit:.2f}')


## 24 hours sim full MR 
To check that the SM and full MR agree we also performed full MR simulations for a supset or the Rep input values used in the SM MR sim

In [ ]:
#import data
Replist = np.array([300,500, 1000, 5000])
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_Rep_{Rep:04d}_slip_velocity.zarr')
#settings
runtime =  timedelta(days=1)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 2, 0, 0, 0, 0)
endtime = starttime + runtime
particle_type = 'inertial_Rep_constant'

chunck_time = 100
nparticles = 88347
data_Rep_constant_24h = {}
for Rep in Replist: 
    file = file_base.format(loc=loc,
                           particle_type = particle_type,
                                                            y_s=starttime.year,
                                                            m_s=starttime.month,
                                                            d_s=starttime.day,
                                                            y_e=endtime.year,
                                                            m_e=endtime.month,
                                                            d_e=endtime.day,
                                                            B = int(B * 1000), 
                                                            tau = int(tau ),
                                                            land_handling = land_handling, 
                                                            cor_on = coriolis,
                                                            Rep = Rep )
    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)
    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_constant_24h[Rep]=ds

In [ ]:
id=1
fig, ax = plt.subplots()
i=0


for Rep, color, marker in zip(Replist,color_array,marker_array):
    ax.plot((data_Rep_constant_24h[Rep].time[id]-data_Rep_constant_24h[Rep].time[id,0])*sec_in_hours,data_Rep_constant_24h[Rep].uf[id],marker,color=color)   


ax.legend(np.char.add('Re$_p$ = ',  Replist.astype(str)))
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [hours]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_c_day={}
bins_Rep_c_day={}
pdfs_Rep_c_day={}
cdfs_Rep_c_day={}

for Rep in Replist:
    Re_particle = Re_slip(data_Rep_constant_24h[Rep].Uslip,diameter, kinematic_viscosity_water)
    array = Re_particle.values.flatten()
    array=array[~np.isnan(array)]

    mean_Rep_c_day[Rep]=np.mean(array)
    
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_c_day[Rep]=bins
    pdfs_Rep_c_day[Rep]=pdf
    cdfs_Rep_c_day[Rep]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for Rep, color, marker in zip(Replist[0:4], color_array, marker_array):
   ax.plot(bins_Rep_c_day[Rep],pdfs_Rep_c_day[Rep],marker,color=color)
ax.legend(np.char.add('Re$_p$ = ',  Replist[0:4].astype(str)))
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
i=0
for Rep in Replist[0:4]:
   
   ax.plot(bins_Rep_c_day[Rep],cdfs_Rep_c_day[Rep],line_array[i],color=color_array[i])
   i+=1 

ax.legend(np.char.add('Re$_p$ = ',  Replist[0:4].astype(str)))
ax.set_xscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# compare with fit and measured values
Repmeasured = [mean_Rep_c_day[Rep] for Rep in Replist]
#plotting
fig, ax = plt.subplots()
# plot measured points

ax.plot(Replist,Repmeasured,'s',color='firebrick')
ax.plot(Replist_SM,Repmeasured_SM,'o',color='navy')
# plot predicted solution based on single value 
Reprange = np.logspace(-1,4,1000)
ax.plot(Reprange,Repmeasured_SM[5] * factor_drag_white1991(Replist_SM[5])/factor_drag_white1991(Reprange),'--',zorder=-5,color='navy')
#legend
ax.legend(['simulated Re$_p^m$ ','simulated Re$_p^m$ SM','predicted Re$_p^m$'])
# plot fitted solution
ax.plot(Reprange,Reprange,color='grey',zorder=-10)
ax.text(85,60,'Re$_p^{in}$ = Re$_p^m$ = '+f'{Rep_fit:.2f}',color='grey',fontsize=15)
# label layout
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(0.1,10**4)
ax.set_ylim(0.1,10**4)
ax.set_xlabel('Re$_p^{in}$')
ax.set_ylabel('Re$_p^{m}$')
# does agree :)

# 24 h  simulation with timevarying drag coefficient
Try time varying drag coefficient with SM MR and full MR 

In [ ]:
# import data
particle_types = ['inertial_SM_drag_Rep','inertial_drag_Rep']
ile_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_Rep_{Rep:04d}_slip_velocity.zarr')
#settings
runtime =  timedelta(days=1)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 2, 0, 0, 0, 0)
endtime = starttime + runtime
particle_type = 'inertial_Rep_constant'
Rep = 457

chunck_time = 100
nparticles = 88347
data_Rep_var_24h = {}
for particle_type in particle_types: 
    file = file_base.format(loc=loc,
                           particle_type = particle_type,
                                                            y_s=starttime.year,
                                                            m_s=starttime.month,
                                                            d_s=starttime.day,
                                                            y_e=endtime.year,
                                                            m_e=endtime.month,
                                                            d_e=endtime.day,
                                                            B = int(B * 1000), 
                                                            tau = int(tau ),
                                                            land_handling = land_handling, 
                                                            cor_on = coriolis,
                                                            Rep = Rep )
    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)
    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_var_24h[particle_type]=ds


In [ ]:

id=1
fig, ax = plt.subplots()

for particle_type, color, marker in zip(particle_types,color_array,marker_array):
    ax.plot((data_Rep_var_24h[particle_type].time[id]-data_Rep_var_24h[particle_type].time[id,0])/sec_in_hours,data_Rep_var_24h[particle_type].uf[id],marker,color=color)   


ax.legend(particle_types)
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [hours]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_var_day={}
bins_Rep_var_day={}
pdfs_Rep_var_day={}
cdfs_Rep_var_day={}

for pt in particle_types:
    Re_particle = Re_slip(data_Rep_var_24h[pt].Uslip,diameter, kinematic_viscosity_water)
    array = Re_particle.values.flatten()
    array=array[~np.isnan(array)]
    array=array[array<100000]

    mean_Rep_var_day[pt]=np.mean(array)
    
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_var_day[pt]=bins
    pdfs_Rep_var_day[pt]=pdf
    cdfs_Rep_var_day[pt]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,marker_array,color_array):
   
   ax.plot(bins_Rep_var_day[pt],pdfs_Rep_var_day[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

# full MR unstable for some partciles s

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,line_array,color_array):
   
   ax.plot(bins_Rep_var_day[pt],cdfs_Rep_var_day[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')

ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')

In [ ]:
for pt in particle_types:
    print(mean_Rep_var_day[pt])

## 1 month simulations constant Rep
1 month sim with both full MR and slow manifold using $\mathrm{Re}_p^{\mathrm{in}} = 457$
and hourly output


In [ ]:
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_Rep_{Rep:04d}_slip_velocity.zarr')

#settings
runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 1, 0, 0, 0, 0)
endtime = starttime + runtime
particle_types = ['inertial_SM_Rep_constant','inertial_Rep_constant']
data_Rep_constant_1month={}
chunck_time = 100
nparticles = 88347
Rep = 457
for pt in particle_types: 
    file = file_base.format(loc=loc,
                           particle_type = pt,
                            y_s=starttime.year,
                            m_s=starttime.month,
                            d_s=starttime.day,
                            y_e=endtime.year,
                            m_e=endtime.month,
                            d_e=endtime.day,
                            B = int(B * 1000), 
                            tau = int(tau ),
                            land_handling = land_handling, 
                            cor_on = coriolis,
                            Rep = Rep )
    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)
    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_constant_1month[pt]=ds

In [ ]:

id=1
fig, ax = plt.subplots()

for pt, color, marker in zip(particle_types,color_array,marker_array):
    ax.plot((data_Rep_constant_1month[pt].time[id]-data_Rep_constant_1month[pt].time[id,0])/sec_in_hours/24,data_Rep_constant_1month[pt].uf[id],marker,color=color)   


ax.legend(particle_types)
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [days]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_c_month={}
bins_Rep_c_month={}
pdfs_Rep_c_month={}
cdfs_Rep_c_month={}
mean_Rep_c_over_month={}

for pt in particle_types:
    Re_particle = Re_slip(data_Rep_constant_1month[pt].Uslip,diameter, kinematic_viscosity_water)
    array = Re_particle.values.flatten()
    array=array[~np.isnan(array)]
    # array=array[array<100000]

    mean_Rep_c_month[pt]=np.mean(array)
    mean_Rep_c_over_month[pt]=Re_particle.mean(dim='trajectory',skipna=True)
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_c_month[pt]=bins
    pdfs_Rep_c_month[pt]=pdf
    cdfs_Rep_c_month[pt]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,marker_array,color_array):
   
   ax.plot(bins_Rep_c_month[pt],pdfs_Rep_c_month[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,line_array,color_array):
   print(mean_Rep_c_month[pt])
   ax.plot(bins_Rep_c_month[pt],cdfs_Rep_c_month[pt],marker,color=color)

ax.legend(particle_types)
ax.set_xscale('log')


ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')


In [ ]:
fig, ax = plt.subplots()
id=1
for pt, marker, color in zip(particle_types,marker_array,color_array):
    ax.plot((data_Rep_constant_1month[pt].time[id]-data_Rep_constant_1month[pt].time[id,0])/sec_in_hours/24,mean_Rep_c_over_month[pt], marker, color=color )

# 1 month varying Rep

In [ ]:
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_slip_velocity.zarr')

#settings
#  /storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_SM_drag_Rep/NWES_start2023_09_01_end2023_10_01_RK4_B0680_tau2759_anti_beaching_cor_True_slip_velocity.zarr.

runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 1, 0, 0, 0, 0)
endtime = starttime + runtime
particle_types = ['inertial_SM_drag_Rep','inertial_drag_Rep']
data_Rep_var_1month={}
chunck_time = 100
nparticles = 88347
for pt in particle_types: 
    
    file = file_base.format(loc=loc,
                           particle_type = pt,
                            y_s=starttime.year,
                            m_s=starttime.month,
                            d_s=starttime.day,
                            y_e=endtime.year,
                            m_e=endtime.month,
                            d_e=endtime.day,
                            B = int(B * 1000), 
                            tau = int(tau ),
                            land_handling = land_handling, 
                            cor_on = coriolis,
                            Rep = Rep )

    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)

    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_var_1month[pt]=ds

    

In [ ]:
# plot slip velocity
id=54634
fig, ax = plt.subplots()

for pt, color, marker in zip(particle_types,color_array,marker_array):
    ax.plot((data_Rep_var_1month[pt].time[id,1:]-data_Rep_var_1month[pt].time[id,0])/sec_in_hours/24,data_Rep_var_1month[pt].uf[id,1:],marker,color=color)   


ax.legend(particle_types)
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [days]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_var_month={}
bins_Rep_var_month={}
pdfs_Rep_var_month={}
cdfs_Rep_var_month={}
mean_Rep_var_over_month={}

for pt in particle_types:
    Re_particle = Re_slip(data_Rep_var_1month[pt].Uslip,diameter, kinematic_viscosity_water)
    array = Re_particle.values.flatten()
    array=array[~np.isnan(array)]
    array=array[array<100000]
    Repmean =Re_particle.where(Re_particle<100000)
    mean_Rep_var_month[pt]=np.mean(array)
    mean_Rep_var_over_month[pt]=Repmean.mean(dim='trajectory', skipna=True)
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_var_month[pt]=bins
    pdfs_Rep_var_month[pt]=pdf
    cdfs_Rep_var_month[pt]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,marker_array,color_array):
   
   ax.plot(bins_Rep_var_month[pt],pdfs_Rep_var_month[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot CDF
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,line_array,color_array):
   
   ax.plot(bins_Rep_var_month[pt],cdfs_Rep_var_month[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')


ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')

In [ ]:
fig, ax = plt.subplots()
id=1
for pt, marker, color in zip(particle_types,marker_array,color_array):
    ax.plot((data_Rep_var_1month[pt].time[id]-data_Rep_var_1month[pt].time[id,0])/sec_in_hours/24,mean_Rep_var_over_month[pt], marker, color=color )

#need small timestepping (30s) for full MR to agree with SM MR

# Measuring Rep in daily averaged field (24 h)
As Rep will on the magnitude of the velocity values in the velocity field and depend on the velocity and derivatives I also measured the slip velocity (and thus the Rep value) in daily detided field to see if this makes a big difference

In [ ]:
file = {}
file['MR'] = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_drag_Rep/NWES_start2023_09_01_tres_daily_end2023_09_02_RK4_B0680_tau2759_anti_beaching_cor_True.zarr'
file['MRSM'] = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_SM_drag_Rep/NWES_start2023_09_01_tres_daily_end2023_09_02_RK4_B0680_tau2759_anti_beaching_cor_True.zarr'
ptypes = ['MR','MRSM']

In [ ]:
chunck_time = 100
nparticles = 88347
ds_daily={}

for pt in ptypes:
    ds = xr.open_dataset(file[pt],
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)

    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    ds_daily[pt]=ds



In [ ]:
id=54634
fig, ax = plt.subplots()

for pt, color, marker in zip(ptypes, color_array,marker_array):

    ax.plot((ds_daily[pt].time[id,1:]-ds_daily[pt].time[id,0])/sec_in_hours,ds_daily[pt].uf[id,1:],marker,color=color)   


ax.legend(['slow manifold MR','full MR'])
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [hours]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_daily_field_24h={}
bins_Rep_daily_field_24h={}
pdfs_Rep_daily_field_24h={}
cdfs_Rep_daily_field_24h={}


for pt in ptypes:
    Re_particle = Re_slip(ds_daily[pt].Uslip,diameter, kinematic_viscosity_water)
    array = Re_particle.values.flatten()
    array=array[~np.isnan(array)]
    array=array[array<100000]
    Repmean =Re_particle.where(Re_particle<100000)
    mean_Rep_daily_field_24h[pt]=np.mean(array)
    # mean_Rep_daily_field_24h[pt]=Repmean.mean(dim='trajectory', skipna=True)
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_daily_field_24h[pt]=bins
    pdfs_Rep_daily_field_24h[pt]=pdf
    cdfs_Rep_daily_field_24h[pt]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(ptypes,marker_array,color_array):
   
   ax.plot(bins_Rep_daily_field_24h[pt],pdfs_Rep_daily_field_24h[pt],marker,color=color)


ax.legend(ptypes)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot CDF
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(ptypes,line_array,color_array):
   
   ax.plot(bins_Rep_daily_field_24h[pt],cdfs_Rep_daily_field_24h[pt],marker,color=color)


ax.legend(ptypes)
ax.set_xscale('log')


ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')

In [ ]:
for pt in ptypes:
    print(mean_Rep_daily_field_24h[pt])
    print(factor_drag_white1991(mean_Rep_daily_field_24h[pt]))